### Initial setup of SEAD staging database

In [ ]:
import sys
sys.path = sys.path + ['.'] if '.' not in sys.path else []
import db

# postgres = db.get_db_engine('postgres')
staging = db.get_db_engine('sead_master_9', host='snares.humlab.umu.se')

%load_ext sql_magic
%config SQL.conn_name = 'staging'
%config SQL.output_result = True
%config SQL.notify_result = True


In [ ]:
%%read_sql -c staging
select ns.nspname, c.relname, d.description
from pg_class c
join pg_description d
  on d.objoid = c.oid
join pg_namespace ns
  on ns.oid = c.relnamespace
where TRUE
  and c.relkind = 'r'
 -- and ns.nspname = 'public'


In [ ]:
%%bash
dbhost=`cat ~/.default.sead.server`
dbuser=`cat ~/.default.sead.username`

sql="\COPY (SELECT * FROM tbl_dating_labs) TO '/tmp/tbl_dating_labs_bugs_20180503.csv' WITH (FORMAT csv, ENCODING 'utf8');"

psql --host=snares.humlab.umu.se --username=$dbuser --no-password --dbname=sead_bugs_import_20180503 --command "$sql"


sql=$(cat <<EOF
    DROP TABLE IF EXISTS tbl_dating_labs_bugs;
    CREATE TABLE tbl_dating_labs_bugs AS SELECT * FROM tbl_dating_labs WHERE FALSE;
EOF
)
psql --host=$dbhost --username=$dbuser --no-password --dbname=sead_staging --command "$sql"

sql="\COPY tbl_dating_labs_bugs FROM '/tmp/tbl_dating_labs_bugs_20180503.csv' WITH (FORMAT csv, ENCODING 'utf8');"
psql --host=$dbhost --username=$dbuser --no-password --dbname=sead_staging --command "$sql"

### Create sead_staging as a copy of sead_master_8
This version of sead_master_8 contains only the public schema.

In [ ]:
%%bash
psql --help

In [ ]:
%%bash
dbhost=`cat ~/.default.sead.server`
dbuser=`cat ~/.default.sead.username`

if [ "$dbhost" != "130.239.1.181" ]; then
    echo "This script can for now only be run on 130.239.1.181";
    exit 64
fi

echo "$dbhost"

sql=$(cat <<EOF
    select pg_terminate_backend(pg_stat_activity.pid)
    from pg_stat_activity 
    where pg_stat_activity.datname in ('sead_staging', 'sead_master_9')
      and pid <> pg_backend_pid();
EOF
)

psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

sql='drop database if exists sead_staging;'
psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

sql='create database sead_staging with template sead_master_9 OWNER sead_master;'
psql --host=$dbhost --username=$dbuser --no-password --dbname=postgres --command "$sql"

### Apply utility change request package

In [ ]:
%%bash
#sqitch tag CS_STAGING_20190517_CREATE_DATABASE --tag v0.01 --note "Create staging database" --plan-file ./utility/sqitch.plan
#sqitch tag --tag v0.1 --note "Initial set of CRs" --plan-file ./utility/sqitch.plan

sqitch deploy --target staging --to @v0.01 --mode change --no-verify -C ./utility/
sqitch deploy --target staging --to @v0.1 --mode change --no-verify -C ./utility/
    

### Apply general change request package

In [ ]:
%%bash
#sqitch tag 20170101_DDL_BIBLIO_REFACTOR_MODEL --tag v0.1 --note "Staging DB with initial DDL changes" --plan-file ./general/sqitch.plan
#sqitch tag 20190513_DML_ECOCODE_ADD_GROUP_UPDATE_SYSTEM --tag v0.2--note "Staging DB with initial DML updates" --plan-file ./general/sqitch.plan

# sqitch deploy --target staging --to @v0.1 --mode change --no-verify -C ./general/
sqitch deploy --target staging --to @v0.2 --mode change --no-verify -C ./general/
    

In [ ]:
%%bash
sqitch help deploy

sqitch deploy --target staging --to v0.1 --mode change --no-verify -C ./utility

In [ ]:
%%read_sql -c postgres
create database sead_staging with template sead_master_9 OWNER sead_master;

In [ ]:

print('hej')